# IMEC2201 Herramientas Computacionales 
## Taller Semana 6: Optimización
### Fecha Límite Entrega (Bloque Neón): <font color="#FF5733">23.59h de Mayo 22, 2022</font>

Universidad de los Andes — Mayo 11, 2022.
___
### Tener en Cuenta
La entrega del Taller 6 se puede realizar de tres maneras:
1. Cargar la solución en su repositorio en GitHub y enviar el enlace del mismo vía Bloque Neón.
2. Enviar vía Bloque Neón el archivo en Jupyter Notebook (extensión '.ipynb').
3. Enviar vía Bloque Neón el archivo PDF del Jupyter Notebook.
___

# Semana 6: Optimizacion

En esta semana se trabajaran diferentes casos de optimización, desde bombas hasta turbinas eólicas. Este tipo de ejercicios se realizan en ingeniería mecánica para el diseño de diferentes sistemas de bombeo o de conversión de energía. En complemento a lo visto en clase, se dejara un taller práctico con el fin de optimizar diferentes parámetros de bombas y turbinas eólicas.

In [1]:
# Importar paquetes
using JuMP
using GLPK
using Ipopt
using Plots
using Dierckx
using DataFrames
using Interpolations
using Roots 
using NLsolve

┌ Info: Precompiling JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1423
┌ Info: Precompiling NLsolve [2774e3e8-f4cf-5e23-947b-6d7e65073b56]
└ @ Base loading.jl:1423


# Bomba Centrifuga

Se necesita conocer la cabeza máxima de una bomba ficticia, cuya eficiencia es constante en un 30%. Opera entre 1000 y 2000 RPM, y el diámetro mínimo es de 10 centímetros. 

**Nota:** Buscar las propiedades hidráulicas del agua a temperatura ambiente.

#### Detalles
Recuerden que la eficiencia de la bomba es la relación entre estos parámetros, es decir:

$$
\eta = \frac{P_w}{P_f} = \frac{\rho g Q H}{w T}
$$

Donde la potencia que la bomba centrífuga le añade al fluido (conocida como **potencia hidráulica** $P_w$), es:

$$
P_w = \rho g Q H
$$

Y para que la bomba centrífuga le añada energía al fluido, debe haber un recurso que, asimismo, le suministre energía al dispositivo. Esto último es conocido como **potencia mecánica** $P_f$ y es el producto entre la velocidad de rotación del eje de la bomba $w$ y el torque en el mismo $T$.

$$
P_f = w T
$$

También:

$$
Q = VA
$$

y

$$
A = \frac{\pi D^2}{4}
$$

In [2]:
function bomba_centrifuga()
    γ = 9800
    rpm_to_rads(omega) = omega*2π/60
    # paso 1:
    model = Model(Ipopt.Optimizer)
    # paso 2:
    @variable(model,rpm_to_rads(1000)<= ω <= rpm_to_rads(2000))
    @variable(model, 0.1 <= d  )
    @variable(model,10<= v)
    @variable(model, η == 0.3)
    @variable(model, 10 <= H)
    @variable(model, T >= 0)
    # paso 3:
    @NLobjective(model, Max, 4*ω*η*T/(γ*π*v*(d^2)))
    # paso 3:
    @NLconstraint(model, c1, η == (γ*v*π*(d^2)*H)/(4* ω* T))
    print(model)
    optimize!(model)
    @show objective_value(model)
    @show value(η)
    @show value(ω)
    @show value(v)
    @show value(H)
    @show value(T)
    @show value(d)
    return objective_value(model)
end

bomba_centrifuga (generic function with 1 method)

In [3]:
bomba_centrifuga()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        5
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       25

Total number of variables............................:        5
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality co

22470.855266751598

# Turbina Eólica

Para un parque eólico en La Guajira, se tomaron medidas de velocidad del viento, las cuales, segun el sitio https://www.windy.com/ oscilan entre 15 y 22 nudos. Si se planea que la velocidad angular de la turbina no supere las 3000 RPM, encontrar el mínimo diámetro posible con el fin de extraer 60 GW de potencia del viento. 

**Nota:**: Tener en cuenta que la potencia eólica es: 

$$
P_w = \frac{1}{2}\rho A v^3
$$

Donde $\rho$ es la densidad del aire.

#### Detalles
Recuerde que:
- 1 nudo equivale a 0.51 m/s.
- La densidad $\rho$ del aire es 1.225 kg/m$^3$.

También, el área $A$ es:

$$
A = \frac{\pi D^2}{4}
$$

In [4]:
function turbina_eolica()
    ms(nudo) = 0.51*nudo
    P = 60e9
    ρ= 1.2
    # paso 1
    model = Model(Ipopt.Optimizer)
    # paso 2
    @variable(model, d >= 0)
    @variable(model,ms(15) <= v <= ms(22))
    # paso 3
    @NLobjective(model, Min, sqrt(8*P/(ρ*π*(v^3))) )
    # paso 4
    @NLconstraint(model, c1, P == 0.5*0.25*ρ*π*(d^2)*(v^3))
    print(model)
    optimize!(model)
    @show objective_value(model)
    @show value(v)
    @show value(d)
    
    return value(d)
end

turbina_eolica (generic function with 1 method)

In [5]:
turbina_eolica()

This is Ipopt version 3.14.4, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        4

Total number of variables............................:        2
                     variables with only lower bounds:        1
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.6746726e+04 6.00e+10 9.88e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00  

9494.361584475739